### Create CSV from midi sources

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import glob
import os
from tqdm import tqdm
from IPython.display import Image, Audio
import traceback

In [3]:
# parallel
from functools import partial
from pathlib import Path

In [4]:
from data_sources import *
from midi_data import *

In [5]:
path = Path('data/midi')
orig_path = path/'midi_sources'
metapath = path/'metadata'
all_csv = metapath/'midi_sources.csv'
metapath.mkdir(parents=True, exist_ok=True)

In [6]:
def create_paths(dirname):
    "Standardize midi_source paths"
    dir_path = orig_path/dirname
    json_path = metapath/f'{dirname}_metadata.json'
    csv_path = metapath/f'{dirname}_metadata.csv'
    return dir_path, json_path, csv_path

In [7]:
sources = ['hooktheory', 'freemidi', 'midiworld', 'ecomp', 'cprato', 'classic_piano', 'wikifonia']

In [8]:
directories = [x for x in path.iterdir() if x.is_dir()]; directories

[PosixPath('data/midi/metadata'),
 PosixPath('data/midi/midi_sources'),
 PosixPath('data/midi/midi_transform_v1'),
 PosixPath('data/midi/midi_pickle_v2'),
 PosixPath('data/midi/midi_transform_v2'),
 PosixPath('data/midi/midi_transform_v2_old'),
 PosixPath('data/midi/midi_transcribe_v2')]

### Remove corrupted file - this causes deadlock with music21 processing

In [19]:
corrf = Path('data/midi/midi_sources/midiworld/named_midi/NITRO_BRO_-_IT_WONT_DIE.mid')
corrf = Path('data/midi/midi_sources/midiworld/named_midi/NITRO_BRO_-_IT_WONT_DIE.mid')

corrupted_files = [
    'midiworld/named_midi/NITRO_BRO_-_IT_WONT_DIE.mid',
    'ecomp/2004/MORET02.mid',
    'ecomp/2006/Mordvinov9.MID',
    'ecomp/2006/Na06.MID',
    'ecomp/2008/Cui01.MID',
    'ecomp/2008/Cui02.MID',
    'ecomp/2008/Cui03.MID',
    'ecomp/2008/Cui04.MID',
    'ecomp/2008/Cui05.MID',
    'ecomp/2008/Cui06.MID',
    'ecomp/2008/Cui07.MID',
    'ecomp/2008/Cui08.MID',
    'ecomp/2008/Tan01.MID',
    'ecomp/2008/Tan02.MID',
    'ecomp/2008/Tan03.MID',
    'ecomp/2018/KaszoS14.MID'
]
for f in corrupted_files:
    fp = path/f
    if fp.exists(): fp.unlink()

### Hooktheory

In [10]:
ht_path, ht_json, ht_csv = create_paths('hooktheory')
ht_song_list = metapath/'hooktheory_key2info.json'

In [11]:
def song_key(s): return '_'.join(s.parts[-3:-1])

In [12]:
ht_key2info = load_json(ht_song_list)

if ht_key2info is None:
    song_info = list((ht_path/'xml').glob('*/*/*/*.json'))
    ht_key2info = {song_key(s):json.load(open(s, 'r')) for s in song_info}
    save_json(ht_key2info, ht_song_list)
    len(song_info)

In [13]:
song_json = list((ht_path/'event').glob('*/*/*/*_key.json')); len(song_json) # using json instead of midi for metadata

19876

In [14]:
def get_ht_midifile(json_file):
    return str(json_file.with_suffix('.mid')).replace('event', 'pianoroll').replace('symbol_', '')

In [15]:
def get_hooktheory_attr(fp):
    song_info = ht_key2info[song_key(fp)]
    song_json = json.load(open(fp, 'r'))
    metadata = song_json['metadata']
    artist = fp.parts[-3]
    title = fp.parts[-2]
    section = fp.name.split('_')[0]
    midi_path = get_ht_midifile(fp)
    
    ht_key = metadata['key']
    ht_mode = metadata['mode']
    if ht_mode is None: ht_mode = 'major'
    ht_offset = keyc_offset(ht_key, ht_mode)
    
    # convert stream here
    metadata = {
        'artist': artist,
        'title': title,
        'midi': midi_path,
        'section': section,
        'parts': song_info['section'],
        'song_url': song_info['song_url'],
        'genres': song_info['genres'],
        'midi_title': metadata['title'],
        'source': 'hooktheory',
        'ht_bpm': metadata['BPM'],
        'ht_mode': metadata['mode'],
        'ht_key': metadata['key'],
        'ht_offset': ht_offset,
        'ht_time_signature': metadata['beats_in_measure']
    }
    
    return {
        'file_path': midi_path, # midi path not json path
        'metadata': metadata
    }

In [16]:
# sanity check
# hook_out = get_hooktheory_attr(song_json[1000]); hook_out

In [17]:
ht_metadata = parse_midi_dir(song_json, ht_json, meta_func=get_hooktheory_attr, key_func=get_ht_midifile)

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/fastprogress/fastprogress.py:95: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


#### Save song_list

In [18]:
ht_metadata = load_json(ht_json)
arr2csv(ht_metadata.values(), ht_csv)
df = pd.read_csv(ht_csv); df.head()

,midi,bpm,source,instruments,artist,song_url,title,inferred_offset,ht_offset,genres,section,seconds,ht_key,time_signature,ht_bpm,parts,inferred_key,ht_time_signature,ht_mode,midi_title
0,data/midi/midi_sources/hooktheory/pianoroll/w/...,128.0,hooktheory,"Piano,Piano",wayne-sharpe,https://www.hooktheory.com/theorytab/view/wayn...,yu-gi-oh-theme-song,0.0,0,NaN,chorus,15.000000,C,4/4,128,"intro,chorus",C major,4,1.0,yu-gi-oh3
1,data/midi/midi_sources/hooktheory/pianoroll/w/...,NaN,hooktheory,NaN,weezer,https://www.hooktheory.com/theorytab/view/weez...,beverly-hills,NaN,0,NaN,intro-and-verse,NaN,C,NaN,128,intro-and-verse,NaN,4,NaN,My New Song
2,data/midi/midi_sources/hooktheory/pianoroll/w/...,108.0,hooktheory,"Piano,Piano",weezer,https://www.hooktheory.com/theorytab/view/weez...,falling-for-you,-3.0,-3,NaN,intro,11.111111,Eb,4/4,108,"intro,verse,chorus,solo",E- major,4,1.0,falling for you intro
3,data/midi/midi_sources/hooktheory/pianoroll/w/...,121.0,hooktheory,"Piano,Piano",weezer,https://www.hooktheory.com/theorytab/view/weez...,buddy-holly,4.0,4,"Pop,Rock",solo,43.388430,Ab,4/4,121,"verse,pre-chorus,chorus,bridge,solo",A- major,4,1.0,Buddy Holly
4,data/midi/midi_sources/hooktheory/pianoroll/w/...,180.0,hooktheory,"Piano,Piano",wavves,https://www.hooktheory.com/theorytab/view/wavv...,dreams-of-grandeur,-2.0,3,NaN,verse,21.333333,A,4/4,180,verse,B minor,4,1.0,dreams of grandeur


In [19]:
df.shape

(19876, 20)

## FreeMidi

In [15]:
fm_path, _, fm_csv = create_paths('freemidi')
fm_dance_path = metapath/f'freemidi_dance_metadata.json'
fm_pop_path = metapath/f'freemidi_pop_metadata.json'
list(fm_path.glob('*'))

[PosixPath('data/midi/midi_sources/freemidi/genre-disco'),
 PosixPath('data/midi/midi_sources/freemidi/genre-pop'),
 PosixPath('data/midi/midi_sources/freemidi/genre-dance-eletric'),
 PosixPath('data/midi/midi_sources/freemidi/genre-punk'),
 PosixPath('data/midi/midi_sources/freemidi/genre-hip-hop-rap'),
 PosixPath('data/midi/midi_sources/freemidi/genre-rock')]

In [19]:
def parse_freemidi_songs(fp, genres=None, source=None):
    name = fp.with_suffix('').name.split(' - ')
    artist = name[0]
    title = name[-1]
    metadata = {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': str(fp),
        'genres': genres,
        'source': source
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [20]:
d_parse_func = partial(parse_freemidi_songs, genres='dance', source='freemidi')
dir_path = fm_path/'genre-dance-eletric'
file_list = get_files(dir_path, extensions=['.mid'], recurse=True)
fm_dance_list = parse_midi_dir(file_list, fm_dance_path, meta_func=d_parse_func)

Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Bjork - Glora.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Tori Amos - Not The Red Baron.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Gloria Estefan - I'm Not Givin' You Up.mid badly formated midi bytes, got: b'RIFFB\x8c\x00\x00RMIDdata~\x8b\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Depeche Mode - Freelove.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Atomic Kitten - Whole Again.mid badly formated midi bytes, got: b'RIFF\x08K\x00\x00RMIDdata{J\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-dance-eletric/Apol

In [21]:
p_parse_func = partial(parse_freemidi_songs, genres='pop', source='freemidi')
dir_path = fm_path/'genre-pop'
file_list = get_files(dir_path, extensions=['.mid'], recurse=True)
fm_pop_list = parse_midi_dir(file_list, fm_pop_path, meta_func=p_parse_func)

Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Cyndi Lauper - Who Let In The Rain.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel?\x13\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Cyndi Lauper - Whats Going On.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel?\x13\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Chris De Burgh - Discovery.mid list index out of range
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Michael Jackson - BilliJeans_AI_composed.mid list index out of range
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Sting - A Thousand Years.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/5th Dimension - One Less Bell To Answere.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/midi_sources/freemidi/genre

Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/ABBA - Anthem.mid badly formated midi bytes, got: b'error with file'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Cyndi Lauper - Shattered Dreams.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel\xfc0\x02\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Anastacia - Stupid Little Things.mid list index out of range
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Gwen Stefani - Early Winter.mid badly formated midi bytes, got: b'<br />\n<b>Warning</b'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Jon Secada - Just Another Day.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/One Direction - Gotta Be You.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Midi Exeption: data/midi/midi_sources/freemid

Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Kinks - Arthur.mid list index out of range
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Wilson Phillips - Release.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Evanescence - Haunted.mid badly formated midi bytes, got: b'RIFF\x0e[\x00\x00RMIDdata\x01[\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Box Car Racer - Cat Like Thief.mid badly formated midi bytes, got: b'RIFF\xd2\x9b\x00\x00RMIDdata\x9b\x9b\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Vapors - Turning Japanese.mid badly formated midi bytes, got: b'\x00\x00\x0ecmoov\x00\x00\x00lmvhd\x00\x00\x00\x00'
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Katy Perry - Hummingbird Heartbeat.mid list index out of range
Midi Exeption: data/midi/midi_sources/freemidi/genre-pop/Fleetwood Mac - Angel.mid index out of range
Midi Exeption: data/midi/midi_sources/free

In [22]:
fmd = load_json(fm_dance_path)
fmp = load_json(fm_pop_path)
fm_all = list(fmd.values())+list(fmp.values())
arr2csv(fm_all, fm_csv)
df = pd.read_csv(fm_csv); df.head()

,seconds,inferred_offset,midi,artist,instruments,inferred_key,genres,bpm,time_signature,source,title
0,28.732394,-5.0,data/midi/midi_sources/freemidi/genre-dance-el...,Tune Up,NaN,F major,dance,142.0,NaN,freemidi,Bounce
1,85.263158,2.0,data/midi/midi_sources/freemidi/genre-dance-el...,Radiohead,"Piano,Piano",G minor,dance,76.0,4/4,freemidi,Fitter Happier
2,40.000000,-4.0,data/midi/midi_sources/freemidi/genre-dance-el...,Daft Punk,"StringInstrument,StringInstrument,StringInstru...",C# minor,dance,102.0,4/4,freemidi,The Grid
3,NaN,NaN,data/midi/midi_sources/freemidi/genre-dance-el...,Bjork,NaN,NaN,dance,NaN,NaN,freemidi,Glora
4,38.400000,3.0,data/midi/midi_sources/freemidi/genre-dance-el...,Enigma,"Clarinet,Clarinet,Guitar,Guitar,Guitar,Guitar,...",A major,dance,300.0,4/4,freemidi,Wanted


### Gather Cprato

In [16]:
cp_path, cp_json, cp_csv = create_paths('cprato')
list(cp_path.glob('*'))[:5]

[PosixPath('data/midi/midi_sources/cprato/Basto - Again And Again (midi By Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/midi_sources/cprato/The Weeknd ft. Lana Del Rey - Stargirl Interlude  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/midi_sources/cprato/Two Steps From Hell - Magic of Love  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/midi_sources/cprato/Bermuda Loverz - My Girl (Ladidada) (Rimini Rockaz Radio Edit) (Midi By Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/midi_sources/cprato/Cascada - Everytime We Touch (Midi By Carlo Prato) (www.cprato.com).mid')]

In [24]:
def parse_cprato_songs(fp, genres=None, source=None):
    name = fp.with_suffix('').name.split(' - ')
    artist = name[0]
    title = name[-1].replace('(midi by Carlo Prato) (www.cprato.com)', '')
    metadata = {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': str(fp),
        'genres': genres,
        'source': source
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [25]:
cp_meta = partial(parse_cprato_songs, genres='EDM,inferred', source='cprato')
file_list = get_files(cp_path, extensions=['.mid'], recurse=True)
cp_md = parse_midi_dir(file_list, cp_json, cp_meta)

In [26]:
cp = load_json(cp_json)
arr2csv(cp.values(), cp_csv)
df = pd.read_csv(cp_csv); df.head()

,seconds,inferred_offset,midi,artist,instruments,inferred_key,genres,bpm,time_signature,source,title
0,28.531469,4,data/midi/midi_sources/cprato/Alex Gaudino - C...,Alex Gaudino,NaN,A- major,"EDM,inferred",143.0,NaN,cprato,Calabria (Drunken Monkey Remix) (Midi By Carlo...
1,45.801527,-1,data/midi/midi_sources/cprato/Oh Wonder - All ...,Oh Wonder,"Flute,Flute,Flute,Piano,Piano,Piano,Fretless B...",C# major,"EDM,inferred",131.0,4/4,cprato,All We Do
2,30.447761,0,data/midi/midi_sources/cprato/Martin Garrix & ...,Martin Garrix & Bebe Rexha,NaN,A minor,"EDM,inferred",134.0,4/4,cprato,In The Name Of Love
3,27.200000,1,data/midi/midi_sources/cprato/Andy SVGE - Evol...,Andy SVGE,"Piano,Piano,Piano,Piano,Piano,Piano",B major,"EDM,inferred",150.0,4/4,cprato,Evolving
4,29.142857,-4,data/midi/midi_sources/cprato/DuMonde - See Th...,DuMonde,"Piano,Piano,Piano,Piano,Piano,Piano",C# minor,"EDM,inferred",140.0,4/4,cprato,See The Light


### Gather MidiWorld

In [17]:
mw_path, mw_json, mw_csv = create_paths('midiworld')

In [12]:
def parse_midiworld_songs(fp):
    name = fp.with_suffix('').name.replace('_', ' ').split(' - ')
    artist = name[0]
    title = name[-1]
    metadata = {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': str(fp),
        'genres': 'pop,inferred',
        'source': 'midiworld'
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [44]:
# Warning: if you get a deadlock: 
# PosixPath('data/midi/midi_sources/midiworld/named_midi/NITRO_BRO_-_IT_WONT_DIE.mid') is broken
file_list = get_files(mw_path/'named_midi', extensions=['.mid'], recurse=True)
mw_md = parse_midi_dir(file_list, mw_json, parse_midiworld_songs)

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/fastprogress/fastprogress.py:95: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [45]:
mw = load_json(mw_json)
arr2csv(mw.values(), mw_csv)
df = pd.read_csv(mw_csv); df.head()

,bpm,title,midi,inferred_offset,genres,artist,seconds,time_signature,inferred_key,source,instruments
0,176.0,Drumloop,data/midi/midi_sources/midiworld/named_midi/Ko...,-2.0,"pop,inferred",Kona,1.193182,4/4,B minor,midiworld,Piano
1,127.0,Millenium,data/midi/midi_sources/midiworld/named_midi/TV...,3.0,"pop,inferred",TV Themes,41.929134,4/4,A major,midiworld,"Timpani,Taiko,Violin,Contrabass,Voice"
2,160.0,Looney Tunes,data/midi/midi_sources/midiworld/named_midi/TV...,0.0,"pop,inferred",TV Themes,19.250000,4/4,C major,midiworld,"Marimba,Acoustic Bass,Trombone,StringInstrumen..."
3,108.0,Diddy Kong,data/midi/midi_sources/midiworld/named_midi/Vi...,-2.0,"pop,inferred",Video Game Themes,73.333333,4/4,B minor,midiworld,"Fretless Bass,Harmonica"
4,105.0,NaN,data/midi/midi_sources/midiworld/named_midi/Ji...,-5.0,"pop,inferred",Jigsaw,104.806742,4/4,D minor,midiworld,"StringInstrument,Horn,Trombone,StringInstrumen..."


In [32]:
p = Path('data/midi/midi_sources/classic_piano/chpn-p20_format0.mid')

In [35]:
p.stat().st_size/1000

2.853

In [31]:
df.midi.iloc[0]

'data/midi/midi_sources/classic_piano/chpn-p20_format0.mid'

### Gather Wikifonia

In [9]:
wf_path, wf_json, wf_csv = create_paths('wikifonia')

In [10]:
def parse_wikifonia_songs(fp):
    name = fp.with_suffix('').name.replace('_', ' ').split(' - ')
    artist = name[0]
    title = name[-1]
    metadata = {
        'artist': artist.strip(),
        'title': title.strip(),
        'xml': str(fp),
        'genres': 'pop,inferred',
        'source': 'wikifonia'
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [22]:
# Warning: if you get a deadlock: 
# PosixPath('data/midi/midi_sources/midiworld/named_midi/NITRO_BRO_-_IT_WONT_DIE.mid') is broken
file_list = get_files(wf_path, extensions=['.mxl'], recurse=True)
wf_md = parse_midi_dir(file_list, wf_json, parse_wikifonia_songs)

Midi Exeption: data/midi/midi_sources/wikifonia/Django Reinhardt - Douce Ambiance.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/midi_sources/wikifonia/Charlie Parker - Scrapple From The Apple.mxl Degree not in specified chord: 9
Midi Exeption: data/midi/midi_sources/wikifonia/Bach ? - Four Part Choralforwikifoniachannel04quant.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/midi_sources/wikifonia/blah3 - blah.mxl failed to get likely keys for Stream component
Midi Exeption: data/midi/midi_sources/wikifonia/Spencer, James F. Hanley - Mary Is Talking.mxl failed to get likely keys for Stream component
Midi Exeption: data/midi/midi_sources/wikifonia/Taj Mahal and Yank Rachel - She Caught The Katy And Left Me A Mule To Ride.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/midi_sources/wikifonia/Johnny Mandel, Johnny Mercer - Emily.mxl local variable 'lyricLanguage' ref

In [23]:
wf = load_json(wf_json)
arr2csv(wf.values(), wf_csv)
df = pd.read_csv(wf_csv); df.head()

,genres,time_signature,seconds,source,artist,quarter_length,bpm,instruments,title,inferred_offset,inferred_key,xml
0,"pop,inferred",4/4,NaN,wikifonia,"Arthur Siegel, June Carroll",76.0,NaN,Voice,Love Is A Simple Thing,0.0,C major,data/midi/midi_sources/wikifonia/Arthur Siegel...
1,"pop,inferred",4/4,NaN,wikifonia,"Henry Mancini, Leslie Bricusse",140.0,NaN,Voice,Two for the Road,5.0,E minor,data/midi/midi_sources/wikifonia/Henry Mancini...
2,"pop,inferred",4/4,NaN,wikifonia,Tom T. Hall,192.0,NaN,NaN,Pamela Brown,-2.0,D major,data/midi/midi_sources/wikifonia/Tom T. Hall -...
3,"pop,inferred",4/4,NaN,wikifonia,"Lucas Secon, Wayne Hector",180.0,NaN,NaN,I Hate This Part,-5.0,F major,"data/midi/midi_sources/wikifonia/Lucas Secon, ..."
4,"pop,inferred",4/4,NaN,wikifonia,"Antonio Carlos Jobim, Gene Lees (English)",144.0,NaN,NaN,Quiet Nights of Quiet Stars,0.0,C major,data/midi/midi_sources/wikifonia/Antonio Carlo...


In [32]:
p = Path('data/midi/midi_sources/classic_piano/chpn-p20_format0.mid')

In [35]:
p.stat().st_size/1000

2.853

In [31]:
df.midi.iloc[0]

'data/midi/midi_sources/classic_piano/chpn-p20_format0.mid'

### Yamaha - piano

In [9]:
ec_path, ec_json, ec_csv = create_paths('ecomp')
ec_song = ec_path/'song_list.json'
list(ec_path.glob('*'))[:5]

[PosixPath('data/midi/midi_sources/ecomp/song_list.json'),
 PosixPath('data/midi/midi_sources/ecomp/2017'),
 PosixPath('data/midi/midi_sources/ecomp/2008'),
 PosixPath('data/midi/midi_sources/ecomp/2006'),
 PosixPath('data/midi/midi_sources/ecomp/2004')]

In [10]:
ec_songs = load_json(ec_path/'song_list.json')

In [11]:
def parse_ecomp_songs(fp):
    song_info = ec_songs[fp.name]
    metadata = {
        'artist': song_info['artist'],
        'title': song_info['title'],
        'midi': str(fp),
        'genres': 'classical',
        'source': 'ecomp'
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [12]:
file_list = get_files(ec_path, extensions=['.mid'], recurse=True)
# file_list = [fp for fp in file_list if fp.stat().st_size/1000 < 200] # over 200mb takes crazy long to analyze
file_list = [fp for fp in file_list if fp.stat().st_size/1000 < 300] # over 200mb takes crazy long to analyze
ec_md = parse_midi_dir(file_list, ec_json, parse_ecomp_songs)

In [13]:
ec = load_json(ec_json)
arr2csv(ec.values(), ec_csv)
df = pd.read_csv(ec_csv); df.head()

,inferred_key,inferred_offset,genres,time_signature,midi,title,seconds,bpm,artist,source,instruments
0,A minor,0.0,classical,4/4,data/midi/midi_sources/ecomp/2017/WangH09.MID,II. Intermezzo in A Minor,212.820513,117.0,,ecomp,Piano
1,F major,-5.0,classical,4/4,data/midi/midi_sources/ecomp/2017/TuanS10.MID,VI. Allegro (Fuga),128.717949,117.0,,ecomp,Piano
2,D minor,-5.0,classical,4/4,data/midi/midi_sources/ecomp/2017/TuanS07.MID,I. Adagio,168.205128,117.0,,ecomp,Piano
3,E major,-4.0,classical,4/4,data/midi/midi_sources/ecomp/2017/MiyashitaM01...,"Prelude and Fugue in E Major, WTC I, BWV 854",150.299145,117.0,Johann Sebastian Bach,ecomp,Piano
4,E- major,-3.0,classical,4/4,data/midi/midi_sources/ecomp/2017/LeeSH13.MID,III. Scherzo: Sehr rasch und markiert,88.547009,117.0,,ecomp,Piano


### Classic Piano

In [27]:
clc_path, clc_json, clc_csv = create_paths('classic_piano')
list(clc_path.glob('*'))[:5]

[PosixPath('data/midi/midi_sources/classic_piano/clementi_opus36_2_2_format0.mid'),
 PosixPath('data/midi/midi_sources/classic_piano/mz_333_2_format0.mid'),
 PosixPath('data/midi/midi_sources/classic_piano/haydn_7_1_format0.mid'),
 PosixPath('data/midi/midi_sources/classic_piano/mendel_op30_4_format0.mid'),
 PosixPath('data/midi/midi_sources/classic_piano/liz_et_trans4_format0.mid')]

In [28]:
def parse_classic_songs(fp):
    name = fp.with_suffix('').name.split('_')
    artist = name[0]
    title = ' '.join(name[1:])
    metadata = {
        'artist': artist,
        'title': title,
        'midi': str(fp),
        'genres': 'classical',
        'source': 'classical_piano'
    }
    return {
        'file_path': fp,
        'metadata': metadata
    }

In [29]:
file_list = get_files(clc_path, extensions=['.mid'], recurse=True)
clc_md = parse_midi_dir(file_list, clc_json, parse_classic_songs, chunk_size=72*4)

In [30]:
clc = load_json(clc_json)
arr2csv(clc.values(), clc_csv)
df = pd.read_csv(clc_csv); df.head()

,bpm,instruments,midi,source,artist,genres,inferred_offset,inferred_key,seconds,title,time_signature
0,41.89,"Piano,Piano",data/midi/midi_sources/classic_piano/chpn-p20_...,classical_piano,chpn-p20,classical,-3,C minor,89.390889,format0,4/4
1,76.39,"Piano,Piano,Piano",data/midi/midi_sources/classic_piano/clementi_...,classical_piano,clementi,classical,0,C major,66.018987,opus36 2 2 format0,3/4
2,52.50,"Piano,Piano",data/midi/midi_sources/classic_piano/chpn-p2_f...,classical_piano,chpn-p2,classical,0,A minor,121.577152,format0,4/4
3,160.00,"Piano,Piano",data/midi/midi_sources/classic_piano/chpn-p10_...,classical_piano,chpn-p10,classical,-4,C# minor,23.307225,format0,3/4
4,88.87,"Piano,Piano,Piano",data/midi/midi_sources/classic_piano/haydn_7_1...,classical_piano,haydn,classical,0,C major,63.082423,7 1 format0,2/4


### Creating CSV

In [9]:
all_csvs = [create_paths(s)[-1] for s in sources]
all_dfs = [pd.read_csv(csv) for csv in all_csvs]

In [10]:
merged_df = pd.concat(all_dfs, sort=False)
merged_df = merged_df.reset_index(drop=True); merged_df

,source,title,midi_title,inferred_offset,inferred_key,ht_offset,ht_mode,artist,seconds,bpm,section,ht_key,instruments,time_signature,ht_time_signature,ht_bpm,song_url,midi,parts,genres
0,hooktheory,yu-gi-oh-theme-song,yu-gi-oh3,0.0,C major,0.0,1.0,wayne-sharpe,15.000000,128.00,chorus,C,"Piano,Piano",4/4,4.0,128.0,https://www.hooktheory.com/theorytab/view/wayn...,data/midi/midi_sources/hooktheory/pianoroll/w/...,"intro,chorus",NaN
1,hooktheory,beverly-hills,My New Song,NaN,NaN,0.0,NaN,weezer,NaN,NaN,intro-and-verse,C,NaN,NaN,4.0,128.0,https://www.hooktheory.com/theorytab/view/weez...,data/midi/midi_sources/hooktheory/pianoroll/w/...,intro-and-verse,NaN
2,hooktheory,falling-for-you,falling for you intro,-3.0,E- major,-3.0,1.0,weezer,11.111111,108.00,intro,Eb,"Piano,Piano",4/4,4.0,108.0,https://www.hooktheory.com/theorytab/view/weez...,data/midi/midi_sources/hooktheory/pianoroll/w/...,"intro,verse,chorus,solo",NaN
3,hooktheory,buddy-holly,Buddy Holly,4.0,A- major,4.0,1.0,weezer,43.388430,121.00,solo,Ab,"Piano,Piano",4/4,4.0,121.0,https://www.hooktheory.com/theorytab/view/weez...,data/midi/midi_sources/hooktheory/pianoroll/w/...,"verse,pre-chorus,chorus,bridge,solo","Pop,Rock"
4,hooktheory,dreams-of-grandeur,dreams of grandeur,-2.0,B minor,3.0,1.0,wavves,21.333333,180.00,verse,A,"Piano,Piano",4/4,4.0,180.0,https://www.hooktheory.com/theorytab/view/wavv...,data/midi/midi_sources/hooktheory/pianoroll/w/...,verse,NaN
5,hooktheory,la-girlz,LA Girlz,4.0,A- major,4.0,1.0,weezer,12.203390,118.00,chorus,Ab,"Piano,Piano",3/4,3.0,118.0,https://www.hooktheory.com/theorytab/view/weez...,data/midi/midi_sources/hooktheory/pianoroll/w/...,"verse,chorus",NaN
6,hooktheory,the-freaking-fcc,Freakin FCC Bridge,0.0,C major,0.0,1.0,walter-murphy,17.142857,126.00,bridge,C,"Piano,Piano",4/4,4.0,126.0,https://www.hooktheory.com/theorytab/view/walt...,data/midi/midi_sources/hooktheory/pianoroll/w/...,"intro-and-verse,bridge",Soundtrack
7,hooktheory,west-wing-suite,snuffy,5.0,G major,5.0,1.0,wg-snuffy-walden,39.069767,86.00,instrumental,G,"Piano,Piano",4/4,4.0,86.0,https://www.hooktheory.com/theorytab/view/wg-s...,data/midi/midi_sources/hooktheory/pianoroll/w/...,instrumental,NaN
8,hooktheory,family-guy-theme-song,Family Guy - Theme Song,-5.0,F major,-5.0,1.0,walter-murphy,32.578125,128.00,verse,F,"Piano,Piano",4/4,4.0,128.0,https://www.hooktheory.com/theorytab/view/walt...,data/midi/midi_sources/hooktheory/pianoroll/w/...,verse,NaN
9,hooktheory,last-christmas,Last Christmas Verse,-1.0,B- minor,-1.0,1.0,wham,35.555556,108.00,verse,Db,"Piano,Piano",4/4,4.0,108.0,https://www.hooktheory.com/theorytab/view/wham...,data/midi/midi_sources/hooktheory/pianoroll/w/...,"intro,verse,chorus",Holiday


In [11]:
[df.shape for df in all_dfs], merged_df.shape

([(19876, 20), (5797, 11), (4715, 11), (2715, 11), (314, 11), (329, 11)],
 (33746, 20))

In [12]:
merged_df.to_csv(all_csv, index=False)